In [7]:
from typing import List, Dict
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from transformers_interpret import MultiLabelClassificationExplainer
from transformers_interpret.explainers.text import PairwiseSequenceClassificationExplainer

In [2]:
def generate_prompts_for_classification(article: str, summary_sentences: List[str]) -> List[Dict]:
    prompts = []
    for sentence in summary_sentences:
        prompt = {"text": article, "text_pair": sentence}
        prompts.append(prompt)
    return prompts

In [ ]:
def predict_with_hf_classification_pipeline(prompts: List[Dict], model_name: str, max_context_length: int = 512,
                                            batch_size: int = 2) -> List[str]:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    text_classification_pipeline = pipeline("text-classification", model=model_name, device=device,
                                            batch_size=batch_size)

    batch_output = text_classification_pipeline(prompts, truncation=True, max_length=max_context_length)
    predictions = [result['label'] for result in batch_output]
    return predictions

In [8]:
model_name = "mtc/mbert-absinth-3-epochs"
# Articles longer than 512 tokens will be truncated
max_context_length = 512

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pairwise_explainer = PairwiseSequenceClassificationExplainer(model, tokenizer)

In [23]:
article = "Ein neuer Zirkus ist gestern in Zürich angekommen. Viele Familien besuchten das grosse Zelt, um die Vorstellung zu sehen. Es gab Akrobaten, Clowns und Tiere, die das Publikum begeisterten. Der Zirkus bleibt noch eine Woche in der Stadt und bietet täglich Vorstellungen an."

summary_sentences = [
    "Ein Zirkus ist in Basel angekommen.",
    "Der Zirkus, der in 1950 gegründet wurde, wird von vielen Familien besucht.",
    "Es gibt tägliche Vorstellungen im Zirkus"]

generated_prompts = generate_prompts_for_classification(article, summary_sentences)

In [24]:
pairwise_attr = pairwise_explainer(article, summary_sentences[2], class_name="Faithful")

In [25]:
pairwise_explainer.visualize("bert_attribution_test_faithful.html")